### TODO

* **!!!Store the public key inside the json file on IPFS!!!**
* Fix Bug: If you don't disconnect from MetaMask and then restart the app, it automatically opens a user page even if the connected MetaMask address is admin
* Decide about the way of encryption / decryption of metadata
* Make a sample json file
* Check that encryption / decryption works well
* Check that only the address which deployed the contract can mint NFTs
* Check what useContext() and createContext() is
* Decide how to handle private key
* Configure copilot

****

### How to deal with Webpack 5 problem

* Context: There are some polyfills necessary for packages that use cryptography but Webpack v5 doesn't include them by default.

The packages not supported by Webpack v5 include:
- buffer
- crypto
- vm
- process
- stream


Solution:

1. Install necessary polyfill packages:

buffer
crypto-browserify
vm-browserify
process
stream-browserify

2. Update webpack.config.js file which is located inside node-modules/react-scripts/config directory if you created your project with react-create-app command

This is what you should add to webpack.config.js:

```
resolve: {
      fallback: {
        "buffer": require.resolve("buffer/"),
        "crypto": require.resolve("crypto-browserify"),
        "vm": require.resolve("vm-browserify"),
        "process": require.resolve('process/browser'),
        "stream": require.resolve('stream-browserify'),
      },
...
plugins: [
    new webpack.ProvidePlugin({
      Buffer: ['buffer', 'Buffer'],
      process: 'process/browser',
    }),
]
```

* Also, add this to index.js

```
import { Buffer } from 'buffer';
import process from 'process';
import { Readable, Writable } from 'stream-browserify';

window.Buffer = Buffer;
window.process = process;
window._Readable = Readable;
window.Writable = Writable;
```

****

### Encrypt with Metamask public key

* https://community.metamask.io/t/how-to-encrypt-message-by-metamask-encryptionpublickey/9746
* https://eips.ethereum.org/EIPS/eip-5630
* eth_decrypt

```
  useEffect(() => {
    if (user && user.length > 0) {
      console.log(user);
      const getPublicKey = async() => {
          try {
              const publicKey = await window.ethereum.request({
                  method: 'eth_getEncryptionPublicKey',
                  params: [user],
              });
              setPubkey(publicKey);
              console.log(publicKey);
          } catch (error) {
              console.error("Failed to fetch public key:", error);
          }
      };
      getPublicKey();
    }
  }, [user]);
```


****

### Storing the contract address permanently

1. Smart Contract Storage:

You could deploy a simple Ethereum smart contract to store contract addresses associated with user accounts. Whenever a user deploys a new contract, this address could be saved to the blockchain through a transaction.
This method is decentralized and ensures that the contract address is accessible from any device as long as you have access to the user's account.


2. Database Storage:

If setting up a smart contract is not ideal, using a centralized database where you can map user accounts to their deployed contract addresses is another practical approach. This method will require a backend to manage user authentication and data retrieval.

****


### Store Private Key Locally

Sic! Local storage means a browser's local storage not a device storage. If you use any other device, there won't be any keys. 

```
// Store private key in local storage (browser example)
localStorage.setItem('privateKey', privateKey);
```

****

### Update navbar after success login or logout redirection

- https://stackoverflow.com/questions/71960194/update-navbar-after-success-login-or-logout-redirection





### React props

* Link: https://www.freecodecamp.org/news/how-to-use-props-in-reactjs/

* Props in React are inputs that you pass into components.
* We can't successfully work with props without having a component to work with.
* React uses a one-way data flow. This means that data can only be transferred from the parent component to the child components. Also, all the data passed from the parent can't be changed by the child component. This means that our data will be passed from App.js which is the parent component to Product.js which is the child component (and never the other way).
* When sending props, you attach your values to them. Below is the syntax:

****

```
<ComponentName property1="value" property2="value" property3="value" />
```

****

**App.js: Parent component**

```
import Product from "./Product";

function App() {
  return (
    <div>
      <h1>PRODUCTS</h1>
      <div className="App">
        <Product
          img="https://ng.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/82/6142201/1.jpg?2933"
          name="Cyxus"
          desc="Non-Slip Fitness Leisure Running Sneakers"
          price="$29"
        />
        <Product
          img="https://ng.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/01/241417/1.jpg?6747"
          name="Vitike"
          desc="Latest Men Sneakers -Black"
          price="$100"
        />
        <Product
          img="https://ng.jumia.is/unsafe/fit-in/300x300/filters:fill(white)/product/06/4410121/1.jpg?4437"
          name="Aomei"
          desc="Men's Trend Casual Sports Shoe"
          price="$40"
        />
      </div>
    </div>
  );
}

export default App;
```

****

**Product.js**

```
//the function receives 'props' as a parameter function
function Product(props) {
    return (
      <div>
//it uses the value of props by defining the parameter as props objects
        <img src={props.img} alt="products" />
        <h4>{props.name}</h4>
        <p>{props.description}</p>
        <h4>{props.price}</h4>
      </div>
    );
}

export default Product
```
****

* **Destructuring**: a feature of JavaScript that involves assigning pieces of data from an object or array to a separate variable so that the variable can hold the data coming from the array or object. To destructure objects in React, **the first step** is to group your properties within a set of curly braces. Then you can either store it into a variable called props within the body of the function or pass it directly as the function’s parameter. **The second step** is to receive the properties where you need them by stating the names of the properties without attaching the prefix ‘props’. 


* Example of desctructuring within the body of the function:

**Product.js**

```
function Product = (props) => {
//First Step: Destructuring within the body of the function
    const { img, name, desc, price} = props ;
    return (
      <div>
  		<img src={img} alt="products" />
//Second Step: receive the properties where you need them by stating the names of the properties without attaching the prefix ‘props.’
        <h4>{name}</h4>
        <p>{description}</p>
        <h4>{price}</h4>
      </div>
    );
}

export default Product
```

* Example of destructuring within the function's parameter:


```
//First Step: Destructuring within function's parameter
function Product = ({ img, name, desc, price}) => {
    return (
      <div>
  		<img src={img} alt="products" />
//Second Step: receive the properties where you need them by stating the names of the properties without attaching the prefix ‘props.’
        <h4>{name}</h4>
        <p>{description}</p>
        <h4>{price}</h4>
      </div>
    );
}

export default Product
```


### Lifting state up

* Link: https://legacy.reactjs.org/docs/lifting-state-up.html

* In React, sharing state is accomplished by moving it up to the closest common ancestor of the components that need it. This is called “lifting state up”. 



### Sharing state between components

* Link: https://react.dev/learn/sharing-state-between-components

* Basically, you should share state by using props.
